# Subspace Model

This notebook illustrate how to use a PPCA model with the [beer framework](https://github.com/beer-asr/beer). 

In [1]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.insert(0, '../')

import beer
import numpy as np
import torch

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
from bokeh.models import ColumnDataSource, LabelSet, Legend
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
import colorcet as cc
output_notebook()

# Convenience functions for plotting.
import plotting

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

## Data

Generate some normally distributed data:

In [2]:
mean = np.array([-10, 10]) 
cov = np.array([[2, 1.5], [1.5, 1.5]])
#cov = np.eye(2) 
data = np.random.multivariate_normal(mean, cov, size=300)

x_range = (mean[0] - 5, mean[0] + 5)
y_range = (mean[1] - 5, mean[1] + 5)

fig = figure(
    title='Data',
    width=400,
    height=400,
    x_range=x_range,
    y_range=y_range
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, mean, cov, alpha=.5, color='Salmon')

show(fig)

## Model Creation

In [3]:
ppca = beer.PPCA.create(torch.zeros(2).double(), 1, torch.eye(1, 2).double())
normal_diag = beer.NormalDiagonalCovariance.create(torch.zeros(2).double(), torch.ones(1).double())
normal_full = beer.NormalFullCovariance.create(torch.zeros(2).double(), torch.eye(2).double())

models = [ppca, normal_diag, normal_full]
parameters = ppca.parameters + normal_diag.parameters + normal_full.parameters

## Variational Bayes Training 

In [4]:
epochs = 30
lrate = 1
X = torch.from_numpy(data).double()
elbo_fn = beer.EvidenceLowerBound(len(X))
optim = beer.BayesianModelOptimizer(parameters)
    
elbos = [[], [], []]
klds = []
for epoch in range(epochs):
    optim.zero_grad()
    for i, model in enumerate(models):
        elbo = elbo_fn(model, X)
        elbo.natural_backward()
        elbos[i].append(round(float(elbo) / len(X), 5))
    optim.step()
    
fig = figure(width=400, height=400)
fig.line(range(epochs), elbos[0], color='blue')
fig.line(range(epochs), elbos[1], color='green')
fig.line(range(epochs), elbos[2], color='red')
show(fig)

In [5]:
colors1 = ['LightBlue']
colors2 = ['LightPink']
values = ['KL', 'LLH']
models = ['Normal (diag. cov.)', 'PPCA', 'Normal (full cov.)']

elbos = [elbo_fn(normal_diag, X), elbo_fn(ppca, X), elbo_fn(normal_full, X), ]
plot_data = {
    'models' : models,
    'labels_orig': [-3, -3, -3],
    'elbos': [
        'ELBO = ' + str(round(float(elbos[0]) / len(X), 3)), 
        'ELBO = ' + str(round(float(elbos[1]) / len(X), 3)), 
        'ELBO = ' + str(round(float(elbos[2]) / len(X), 3))
    ],
    'KL': [
        float(elbos[0].kl_div) / len(X),
        float(elbos[1].kl_div) / len(X),
        float(elbos[2].kl_div) / len(X)
    ],
    'LLH': [
        float(elbos[0].expected_llh) / len(X),
        float(elbos[1].expected_llh) / len(X),
        float(elbos[2].expected_llh) / len(X)
    ]
}
source = ColumnDataSource(data=plot_data)

fig1 = figure(y_range=models, x_range=(-4, 2), plot_height=250, plot_width=550,
              )
p_kld = fig1.hbar_stack(['KL'], y='models', height=.8, line_width=0.01, color=colors1, 
                        muted_color='grey', source=source)
p_llh = fig1.hbar_stack(['LLH'], y='models', height=.8, line_width=0.01, color=colors2, 
                        muted_color='grey', source=source)

#source_txt = ColumnDataSource(data=dict(models=models, elbos=[0, 1, 2]))
labels = LabelSet(x='LLH', y='models', text='elbos', x_offset=10., y_offset=-9, source=source)
fig1.add_layout(labels)

legend = Legend(items=[
        ('KL divergence ', p_kld), 
        ('exp. log-likelihood ', p_llh)
    ], location=(10, 150))
legend.click_policy='mute'
fig1.add_layout(legend, 'right')

show(fig1)

In [6]:
elbos[0]._local_kl_div

tensor(0., dtype=torch.float64)

In [8]:
model = ppca

x = np.linspace(-20, 20, 1000)

A, B = model.mean.numpy(), model.mean.numpy() + model.subspace.numpy()[0, :]
slope = (A[1] - B[1]) / (A[0] - B[0])
intercept = -slope * ((slope * A[0] - A[1]) / slope)
s_line = np.c_[x, slope * x + intercept]
p_h = np.sqrt(1 / (2 * np.pi)) * np.exp(-.5 * x ** 2)
angle = np.arctan(abs(B[1] - A[1]) / abs(B[0] - A[0]))
R = np.array([
    [np.cos(angle), -np.sin(angle)],
    [np.sin(angle), np.cos(angle)]
])

fig1 = figure(
    x_range=x_range,
    y_range=y_range,
    width=400,
    height=400,
)

plotting.plot_normal(fig1, mean, cov, alpha=.4, color='Salmon')

for i in range(2):
    xy = np.c_[x, np.zeros_like(x)]
    rxy1 = xy @ R.T + model.mean.numpy()
    xy = np.c_[x, (i + 1) * np.sqrt(np.ones_like(x) / model.precision.numpy())]
    rxy2 = xy @ R.T + model.mean.numpy()
    band_x = np.append(rxy1[:,0], rxy2[:, 0][::-1])
    band_y = np.append(rxy1[:,1], rxy2[:, 1][::-1])
    fig1.patch(band_x, band_y, line_alpha=0., fill_alpha=0.5, fill_color='LightBlue')

    xy = np.c_[x, np.zeros_like(x)]
    rxy1 = xy @ R.T + model.mean.numpy()
    xy = np.c_[x, -(i + 1) * np.sqrt(np.ones_like(x) / model.precision.numpy())]
    rxy2 = xy @ R.T + model.mean.numpy()
    band_x = np.append(rxy1[:,0], rxy2[:, 0][::-1])
    band_y = np.append(rxy1[:,1], rxy2[:, 1][::-1])
    fig1.patch(band_x, band_y, line_alpha=0., fill_alpha=0.5, fill_color='LightBlue')


xy = np.c_[x, np.zeros_like(x)]
rxy1 = xy @ R.T + mean
xy = np.c_[x, p_h]
rxy2 = xy @ R.T + mean
band_x = np.append(rxy1[:,0], rxy2[:, 0][::-1])
band_y = np.append(rxy1[:,1], rxy2[:, 1][::-1])
fig1.patch(band_x, band_y, line_color='black', fill_color='LightGreen')

fig2 = figure(
    width=400,
    height=400,
    x_range=x_range,
    y_range=y_range
)
fig2.cross(data[:, 0], data[:, 1])
plotting.plot_normal(fig2, mean, cov, alpha=.5, color='Salmon')

show(gridplot([[fig2, fig1]]))